# Examine phenometric trends along various gradients

* P:PET
* P_annual_sd/P_mean_annual
* NDVI_mean_annual
* Woody cover fraction
* Map the trends in VPD and see how they compare with the switch in trends between 1982-2000, and 2000-2022
    * From sami's paper: VPD changed minimally between 1982–2000, whereas it rapidly increased between 2001–2019 (Fig. A8). These increases were largest in the most Arid and Temperate regions (12.7 % and 11 % since 1982, respectively; Fig. 6), and when coupled with seasonal reductions in precipitation (Fig. A2) these would have partially offset benefits from increased intrinsic water use efficiency (Eq. 17). 

In [ ]:
%matplotlib inline

import sys
import pickle
import warnings
import odc.geo.xr
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sb
from scipy import stats
import geopandas as gpd
import pymannkendall as mk
import matplotlib.pyplot as plt
from odc.geo.geom import Geometry


## Analysis Parameters


In [ ]:
# product='GIMMS-PKU'
product='AusENDVI-clim'
# product='AusENDVI-clim_MCD43A4'
# product='AusENDVI-clim_2000'
# product='GIMMS-PKU_2022'
# product='GIMMSv1.1'
# product='GIMMSv1.2'
# product='MCD43A4'
# product='Landsat'

timeseries_file = '/g/data/os22/chad_tmp/Aus_phenology/data/pickle/IBRA_subregions_NDVI_'+product+'.pkl'
phenometrics_file = '/g/data/os22/chad_tmp/Aus_phenology/data/pickle/IBRA_subregions_'+product+'_phenometrics.pkl'
climate_file = '/g/data/os22/chad_tmp/Aus_phenology/data/pickle/IBRA_subregions_climate.pkl'
ecoregions_file = '/g/data/os22/chad_tmp/Aus_phenology/data/vectors/IBRAv7_subregions_modified.geojson'

var='SUB_NAME_7'

## Open data

* Penman PET from AWRA: '/g/data/os22/chad_tmp/NEE_modelling/data/5km/PET_5km_monthly_2002_2021.nc'
* Pan evaporation from ANUClim: '/g/data/os22/chad_tmp/NEE_modelling/data/1km/evap_1km_monthly_2002_2021.nc'
* MI: '/g/data/os22/chad_tmp/NEE_modelling/data/5km/MI_5km_monthly_2002_2021.nc'
* AI: '/g/data/os22/chad_tmp/NEE_modelling/data/5km/AridityIndex_5km_monthly_2002_2021.nc'

In [ ]:
gdf = gpd.read_file(ecoregions_file)

with open(climate_file, 'rb') as f:
    climate = pickle.load(f)

with open(timeseries_file, 'rb') as f:
    ndvi = pickle.load(f)

with open(phenometrics_file, 'rb') as f:
    phenometrics = pickle.load(f)

mi = xr.open_dataarray('/g/data/os22/chad_tmp/NEE_modelling/data/5km/MI_5km_monthly_2002_2021.nc')
mi = mi.mean('time')
mi = mi.rename('P:PET')
mi = xr.where(mi>1.5, 1.5, mi) #remove extreme values

wcf = xr.open_dataarray('/g/data/os22/chad_tmp/AusENDVI/data/5km/WCF_5km_monthly_1982_2022.nc')
wcf = wcf.mean('time')

vegh = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/data/5km/VegH_5km_monthly_1982_2022.nc')['VegH']
vegh = vegh.mean('time')

# trees = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/data/5km/trees_5km_monthly_1982_2022.nc')['trees']
# trees = trees.mean('time')

## Remove all cropping regions

In [ ]:
ndvi = {k:v for k, v in ndvi.items() if not 'crop' in k}
climate = {k:v for k, v in climate.items() if not 'crop' in k}
phenometrics = {k:v for k, v in phenometrics.items() if not 'crop' in k}

## Extract trends and environmental gradients


In [ ]:
trends_dfs = []
pheno_stats=['SOS','vSOS','LOS','POS','vPOS','AOS',
             'EOS', 'vTOS','vEOS','ROG','ROS', 'IOS']
i=0
for k in phenometrics.keys():
    print("Feature {:02}/{:02}\r".format(i + 1, len(range(0, len(phenometrics)))), end="")

    #get data for the polygon
    df = phenometrics[k]
    clim = climate[k].sel(time=slice('1982','2013'))
    veg = ndvi[k]

    #mask the wcf and mi datasets
    geom = Geometry(geom=gdf[gdf[var]==k].geometry.item(), crs=gdf.crs)
    moist = mi.odc.mask(poly=geom)
    woody = wcf.odc.mask(poly=geom)
    height = vegh.odc.mask(poly=geom)
    
    #now calculate the enviro gradients
    #rain variability
    annual_rain = clim['rain'].resample(time='YE').sum()
    rain_variability = annual_rain.std('time').values.item() / annual_rain.mean('time').values.item()
    
    #mean NDVI
    annual_mean_veg = veg.resample(time='YE').mean().mean('time').values.item()

    #mean WCF
    woody = woody.mean().values.item()

    #mean P:PET
    moist = moist.mean().values.item()

    #veg height
    height = height.mean().values.item()

    # loop through phen stats and append
    # enviro info
    trends = {}
    for pheno in pheno_stats:

        trends['annual_mean_NDVI'] = annual_mean_veg
        trends['rainfall_variability'] = rain_variability
        trends['P_PET'] = moist
        trends['WCF'] = woody
        trends['VegHeight'] = height
        
        result = mk.original_test(phenometrics[k][pheno], alpha=0.05)
        # result = stats.linregress(x=df.index, y=df[pheno])
        trends[pheno+'_p-value'] = result.p
        trends[pheno+'_slope'] = result.slope
        trends[pheno+'_intercept'] = result.intercept
        trends[var] = k
        
    trend_df=pd.DataFrame(trends, index=range(0,1))
    trends_dfs.append(gdf.merge(trend_df, on=var))
    i+=1

pheno_trends = pd.concat(trends_dfs).reset_index(drop=True)
pheno_trends = pheno_trends.dropna() #there's one nan by the coast in the north for P:PET

### Plot all variables


In [ ]:
pheno_stats=['SOS_slope','vSOS_slope', 'LOS_slope', 
             'POS_slope', 'vPOS_slope', 'ROG_slope',
             'EOS_slope', 'vEOS_slope', 'ROS_slope',
             'AOS_slope', 'vTOS_slope', 'IOS_slope'
            ]

for variable in ['annual_mean_NDVI', 'WCF', 'rainfall_variability', 'P_PET', 'VegHeight']:
    
    with plt.style.context('ggplot'):
        fig,axes=plt.subplots(4,3, figsize=(10,10),  layout='constrained', sharex=True)
        
        for ax,pheno in zip(axes.ravel(), pheno_stats):
            obs,pred = pheno_trends[variable].values, pheno_trends[pheno].values
            sl, i, r, p, s = stats.linregress(obs,pred)
            r2 = r**2
            
            sb.scatterplot(data=pheno_trends, x=variable,y=pheno, alpha=0.7, ax=ax, color='tab:blue')
            sb.regplot(data=pheno_trends, x=variable,y=pheno, scatter=False,
                       line_kws={'linewidth':1}, color='black', ax=ax)
            
            ax.axhline(0, color='grey', linestyle='--')
            ax.text(.70, .90, 'r\N{SUPERSCRIPT TWO}={:.2f}'.format(r2),
                    transform=ax.transAxes, fontsize=10)
            ax.text(.70, .825, 'p={:.3f}'.format(p), transform=ax.transAxes, fontsize=10)
    
            fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/Trends_{variable}_{product}.png',
                                        bbox_inches='tight', dpi=300)

## Trends in VPD

In [ ]:
from xarrayMannKendall import Mann_Kendall_test

In [ ]:
vpd = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/data/5km/vpd_5km_monthly_1982_2022.nc')['vpd']
vpd = vpd.resample(time='YE').mean()

In [ ]:
vpd.mean(['latitude', 'longitude']).plot(figsize=(12,4))
plt.ylabel('annual mean VPD')

In [ ]:
vpd_trend_2000 = Mann_Kendall_test(vpd.sel(time=slice('1982','1999')), alpha=0.05, method='theilslopes',
                             coords_name={'time':'time','longitude':'x','latitude':'y'}).compute()

vpd_trend_2022 = Mann_Kendall_test(vpd.sel(time=slice('2000','2022')), alpha=0.05, method='theilslopes',
                             coords_name={'time':'time','x':'longitude','y':'latitude'}).compute()

In [ ]:
vpd_trend_2000.trend.plot(size=5, robust=True, vmin=-0.2, vmax=0.2, cmap='RdBu_r')
plt.title('VPD trend 1982-1999')

In [ ]:
vpd_trend_2022.trend.plot(size=5, robust=True, vmin=-0.2, vmax=0.2, cmap='RdBu_r')
plt.title('VPD trend 2000-2022')